In [2]:
import pandas as pd

In [3]:
import logging
import os
import os.path as op

def generate_lineidx_file(filein, idxout):
    idxout_tmp = idxout + '.tmp'
    with open(filein, 'r') as tsvin, open(idxout_tmp,'w') as tsvout:
        fsize = os.fstat(tsvin.fileno()).st_size
        fpos = 0
        while fpos!=fsize:
            tsvout.write(str(fpos)+"\n")
            tsvin.readline()
            fpos = tsvin.tell()
    os.rename(idxout_tmp, idxout)

class TSVFile(object):
    def __init__(self, tsv_file, generate_lineidx=False):
        self.tsv_file = tsv_file
        self.lineidx = op.splitext(tsv_file)[0] + '.lineidx'
        self._fp = None
        self._lineidx = None
        # the process always keeps the process which opens the file. 
        # If the pid is not equal to the currrent pid, we will re-open the file.
        self.pid = None
        # generate lineidx if not exist
        if not op.isfile(self.lineidx) and generate_lineidx:
            generate_lineidx_file(self.tsv_file, self.lineidx)

    def __del__(self):
        if self._fp:
            self._fp.close()

    def __str__(self):
        return "TSVFile(tsv_file='{}')".format(self.tsv_file)

    def __repr__(self):
        return str(self)

    def num_rows(self):
        self._ensure_lineidx_loaded()
        return len(self._lineidx)

    def seek(self, idx):
        self._ensure_tsv_opened()
        self._ensure_lineidx_loaded()
        try:
            pos = self._lineidx[idx]
        except:
            logging.info('{}-{}'.format(self.tsv_file, idx))
            raise
        self._fp.seek(pos)
        return [s.strip() for s in self._fp.readline().split('\t')]

    def seek_first_column(self, idx):
        self._ensure_tsv_opened()
        self._ensure_lineidx_loaded()
        pos = self._lineidx[idx]
        self._fp.seek(pos)
        print(self._fp)
        return read_to_character(self._fp, '\t')

    def __getitem__(self, index):
        return self.seek(index)

    def __len__(self):
        return self.num_rows()

    def _ensure_lineidx_loaded(self):
        if self._lineidx is None:
            logging.info('loading lineidx: {}'.format(self.lineidx))
            with open(self.lineidx, 'r') as fp:
                self._lineidx = [int(i.strip()) for i in fp.readlines()]

    def _ensure_tsv_opened(self):
        if self._fp is None:
            self._fp = open(self.tsv_file, 'r')
            self.pid = os.getpid()

        if self.pid != os.getpid():
            logging.info('re-open {} because the process id changed'.format(self.tsv_file))
            self._fp = open(self.tsv_file, 'r')
            self.pid = os.getpid()

In [ ]:
img = TSVFile("visualgenome/img.tsv")
img.seek(1)

In [10]:
hw = TSVFile("visualgenome/hw.tsv")
hw.seek(1)

['1', '[{"height": 600, "width": 800}]']

In [44]:
label = TSVFile("visualgenome/label_danfeiX_clipped.new.tsv")
ret = label.seek(1)
print(eval(ret[1]))

{'objects': [{'class': 'sidewalk', 'rect': [320, 347, 798, 600], 'attributes': ['brick', 'white'], 'attributes_encode': [37, 384]}, {'class': 'building', 'rect': [569, 0, 797, 414], 'attributes': ['brown', 'tall', 'orange'], 'attributes_encode': [41, 357, 224]}, {'class': 'building', 'rect': [171, 0, 429, 319], 'attributes': ['red', 'brown'], 'attributes_encode': [273, 41]}, {'class': 'man', 'rect': [321, 325, 463, 571], 'attributes': ['walking', 'white'], 'attributes_encode': [377, 384]}, {'class': 'pole', 'rect': [423, 0, 472, 533], 'attributes': ['brown'], 'attributes_encode': [41]}, {'class': 'window', 'rect': [649, 68, 743, 233], 'attributes': ['glass'], 'attributes_encode': [137]}, {'class': 'car', 'rect': [240, 354, 367, 462], 'attributes': ['red', 'parked'], 'attributes_encode': [273, 235]}, {'class': 'tree', 'rect': [0, 146, 89, 366], 'attributes': ['green'], 'attributes_encode': [149]}, {'class': 'tree', 'rect': [102, 195, 208, 356], 'attributes': ['green'], 'attributes_encod

In [ ]:
val = TSVFile("VizWiz/val.tsv")
val.seek(1)

In [46]:
val_label = TSVFile("VizWiz/val.label.tsv")
val_label.seek(1)

['VizWiz_val_00002849',
 '[{"rect": [1, 1, 30, 40], "class": "Dog"}, {"rect": [2, 3, 100, 100], "class": "Cat"}]']

In [47]:
val_hw = TSVFile("VizWiz/val.hw.tsv")
val_hw.seek(1)

['VizWiz_val_00002849', '[{"height": 2048, "width": 1536}]']

In [41]:
import base64
import numpy as np
pred = TSVFile("../output/X152C5_test/inference/vinvl_vg_x152c4/predictions.tsv")
ret = pred.seek(1)
img = ret[0]
fields = eval(ret[1].replace('null', 'None'))
# print(fields)
feature = base64.b64decode(fields['objects'][0]['feature'])
print(np.frombuffer(feature, np.float32).shape)

(2048,)


In [23]:
from ..maskrcnn_benchmark.utils.env import setup_environment

import torch
from ..maskrcnn_benchmark.config import cfg
from ..scene_graph_benchmark.config import sg_cfg
from ..maskrcnn_benchmark.data import make_data_loader
from ..maskrcnn_benchmark.data.datasets.utils.load_files import config_dataset_file
from ..maskrcnn_benchmark.engine.inference import inference
from ..scene_graph_benchmark.scene_parser import SceneParser
from ..scene_graph_benchmark.AttrRCNN import AttrRCNN
from ..maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from ..maskrcnn_benchmark.utils.collect_env import collect_env_info
from ..maskrcnn_benchmark.utils.comm import synchronize, get_rank
from ..maskrcnn_benchmark.utils.logger import setup_logger
from ..maskrcnn_benchmark.utils.miscellaneous import mkdir

pred_pt = torch.load("../output/X152C5_test/inference/vinvl_vg_x152c4/predictions.pth")
print(type(pred_pt))

ImportError: attempted relative import with no known parent package

ModuleNotFoundError: No module named 'maskrcnn_benchmark'